In [ ]:
%%capture
!pip install evaluate
!pip install transformers

In [ ]:
import subprocess

def get_gpu_name():
    try:
        output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], encoding='utf-8')
        gpu_name = output.strip()
        return gpu_name
    except (subprocess.CalledProcessError, FileNotFoundError):
        return None
print(get_gpu_name())

Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import evaluate
import re

from enum import Enum
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForPreTraining
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/violence/violence-v1.csv")

In [ ]:
print(len(df))
df = df[df['text'].notna()]
print(len(df))

8148
8147


In [ ]:
class ViolenceType(Enum):
    DEROGATION = 1
    ANTIPATHY = 2
    PREJUDICATION = 3
    REPRESSION = 4
    
class Annotation:
    def __init__(self, text, religio, ethno, nondenominational, noncommunal):
        self.text = text
        self.religio = religio
        self.ethno = ethno
        self.nondenominational = nondenominational
        self.noncommunal = noncommunal

In [ ]:
def categorize_violence(violences, text):
    ore = [violences]
    violences = str(violences)
    violences = re.findall(r'\d',violences)
    return_list = list()
    for v in violences:
        if int(v)!=0:
            return_list.append(int(v))
    return return_list

In [ ]:
annotation_list = []
for idx, row in df.iterrows():
    text = row.text
    annotation = Annotation(
        row.text,
        categorize_violence(row['religio-communal'],text),
        categorize_violence(row['ethno-communal'], text),
        categorize_violence(row['other-communal'], text),
        categorize_violence(row['generic-violence'], text)
    )
    annotation_list.append(annotation)

In [ ]:
for idx, row in df.iterrows():
    if not isinstance(row.text, str) :
        print(row)

In [ ]:
class AnnotationSixteen:
    def __init__(self,text):
        self.text = text
        self.religio_derogation = False
        self.religio_antipathy = False
        self.religio_prejudication = False
        self.religio_repression = False
        self.ethno_derogation = False
        self.ethno_antipathy = False
        self.ethno_prejudication = False
        self.ethno_repression = False
        self.nondenominational_derogation = False
        self.nondenominational_antipathy = False
        self.nondenominational_prejudication = False
        self.nondenominational_repression = False
        self.noncommunal_derogation = False
        self.noncommunal_antipathy = False
        self.noncommunal_prejudication = False
        self.noncommunal_repression = False

annotation_sixteen_list = list()
list_a = ['religio','ethno','nondenominational','noncommunal']
list_b = ['derogation','antipathy','prejudication','repression']

for i in annotation_list:
    annotation = AnnotationSixteen(i.text)
    if len(i.religio)>0:
        for j in i.religio:
            setattr(annotation, f"religio_{list_b[j-1]}",True)
    if len(i.ethno)>0:
        for j in i.ethno:
            setattr(annotation, f"ethno_{list_b[j-1]}",True)
    if len(i.nondenominational)>0:
        for j in i.nondenominational:
            setattr(annotation, f"nondenominational_{list_b[j-1]}",True)
    if len(i.noncommunal)>0:
        for j in i.noncommunal:
            setattr(annotation, f"noncommunal_{list_b[j-1]}",True)
    annotation_sixteen_list.append(annotation)
            
tempdf_v2 = pd.DataFrame([x.__dict__ for x in annotation_sixteen_list])

In [ ]:
tempdf_v2.head()

,text,religio_derogation,religio_antipathy,religio_prejudication,religio_repression,ethno_derogation,ethno_antipathy,ethno_prejudication,ethno_repression,nondenominational_derogation,nondenominational_antipathy,nondenominational_prejudication,nondenominational_repression,noncommunal_derogation,noncommunal_antipathy,noncommunal_prejudication,noncommunal_repression
0,দাদা নাম শুনে বিন্দাবনের টিকিট কাটছি টাকাটা দি...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,"সবই সিষ্টেম রে ভাই, সবই সিষ্টেম। এই বেডারে হাই...",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,দিদিমনি কতবার কেন্দ্র কে বর্ডার সুরক্ষা নিয়ে ক...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,"সালাম সেই যুদ্ধাদের প্রতি, যাদের বীরত্বের জন্...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,বাল পাকনা চেতনাবাজ গুলো আছে চঞ্চল চৌধুরী নিয়ে ...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [ ]:
from datasets import DatasetDict

dataset = Dataset.from_pandas(tempdf_v2)
train_test_valid = dataset.train_test_split(test_size=0.4)
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'religio_derogation', 'religio_antipathy', 'religio_prejudication', 'religio_repression', 'ethno_derogation', 'ethno_antipathy', 'ethno_prejudication', 'ethno_repression', 'nondenominational_derogation', 'nondenominational_antipathy', 'nondenominational_prejudication', 'nondenominational_repression', 'noncommunal_derogation', 'noncommunal_antipathy', 'noncommunal_prejudication', 'noncommunal_repression'],
        num_rows: 4888
    })
    test: Dataset({
        features: ['text', 'religio_derogation', 'religio_antipathy', 'religio_prejudication', 'religio_repression', 'ethno_derogation', 'ethno_antipathy', 'ethno_prejudication', 'ethno_repression', 'nondenominational_derogation', 'nondenominational_antipathy', 'nondenominational_prejudication', 'nondenominational_repression', 'noncommunal_derogation', 'noncommunal_antipathy', 'noncommunal_prejudication', 'noncommunal_repression'],
        num_rows: 1630
    })
    valid: Da

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['religio_derogation',
 'religio_antipathy',
 'religio_prejudication',
 'religio_repression',
 'ethno_derogation',
 'ethno_antipathy',
 'ethno_prejudication',
 'ethno_repression',
 'nondenominational_derogation',
 'nondenominational_antipathy',
 'nondenominational_prejudication',
 'nondenominational_repression',
 'noncommunal_derogation',
 'noncommunal_antipathy',
 'noncommunal_prejudication',
 'noncommunal_repression']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")

In [ ]:
def preprocess_data(examples):
  text = examples["text"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  labels_matrix = np.zeros((len(text), len(labels)))
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/4888 [00:00<?, ? examples/s]

Map:   0%|          | 0/1630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
encoded_dataset.set_format("torch")
print(encoded_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4888
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1630
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1629
    })
})


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "csebuetnlp/banglabert", 
    problem_type="multi_label_classification", 
    num_labels=len(labels), 
    id2label=id2label, 
    label2id=label2id
)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
metric_name = "f1"

training_args = TrainingArguments(
    output_dir="violence",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.143856,0.000000,0.500000,0.472069
2,0.199100,0.138817,0.235925,0.572017,0.511971
3,0.199100,0.129989,0.232495,0.570213,0.511357
4,0.123100,0.128763,0.294306,0.594635,0.517495
5,0.104300,0.129734,0.272606,0.584689,0.516882
6,0.104300,0.130282,0.317654,0.606506,0.515040
7,0.084000,0.134456,0.377944,0.643275,0.497851
8,0.084000,0.141034,0.400983,0.664409,0.480663
9,0.068700,0.153729,0.391825,0.658313,0.487416
10,0.053800,0.158908,0.409341,0.678781,0.457336


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.143856,0.000000,0.500000,0.472069
2,0.199100,0.138817,0.235925,0.572017,0.511971
3,0.199100,0.129989,0.232495,0.570213,0.511357
4,0.123100,0.128763,0.294306,0.594635,0.517495
5,0.104300,0.129734,0.272606,0.584689,0.516882
6,0.104300,0.130282,0.317654,0.606506,0.515040
7,0.084000,0.134456,0.377944,0.643275,0.497851
8,0.084000,0.141034,0.400983,0.664409,0.480663
9,0.068700,0.153729,0.391825,0.658313,0.487416
10,0.053800,0.158908,0.409341,0.678781,0.457336


TrainOutput(global_step=5508, training_loss=0.07153615914420451, metrics={'train_runtime': 9149.9879, 'train_samples_per_second': 16.026, 'train_steps_per_second': 1.003, 'total_flos': 2.315247300299981e+16, 'train_loss': 0.07153615914420451, 'epoch': 18.0})

In [ ]:
trainer.evaluate(encoded_dataset["test"])

{'eval_loss': 0.18914318084716797,
 'eval_f1': 0.41171251109139306,
 'eval_roc_auc': 0.6869023213819666,
 'eval_accuracy': 0.4699386503067485,
 'eval_runtime': 58.5759,
 'eval_samples_per_second': 27.827,
 'eval_steps_per_second': 1.741,
 'epoch': 18.0}

In [ ]:
encoded_dataset.save_to_disk("/content/drive/MyDrive/violence/violence-v1-16.hf")


Saving the dataset (0/1 shards):   0%|          | 0/4888 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1630 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1629 [00:00<?, ? examples/s]

In [ ]:
trainer.save_model("/content/drive/MyDrive/violence/violence-sixteen-model")
